<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/AGENT_ORCHESTRARION_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet



#FlashAttention only supports Ampere GPUs or newer. #NEED A100 OR L4 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

!pip install crewai --quiet
!pip install langchain --quiet
!pip install faiss-gpu --quiet
!pip install sentence-transformers --quiet

!pip install langchain_community -q

!pip install faiss-cpu -q

!pip install crewai-tools -q

!pip install transformers -U -q

In [ ]:
!nvidia-smi

Sat Mar 22 15:50:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   43C    P8             17W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.callbacks.manager import CallbackManager
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import warnings
warnings.filterwarnings("ignore")

import colab_env

Mounted at /content/gdrive


## new code

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install colab-env --quiet
!pip install langchain_community -q
!pip install faiss-cpu -q
!pip install crewai-tools -q
!pip install transformers -U -q

In [8]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install colab-env --quiet
!pip install langchain_community -q
!pip install faiss-cpu -q
!pip install crewai-tools -q
!pip install transformers -U -q

In [ ]:
!pip install crewai==0.28.8 -q
!pip crewai_tools==0.1.6 -q
!pip langchain_community==0.0.29 -q
!pip install transformers -U -q

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import HuggingFaceHub

In [1]:
!pip install colab-env --quiet
import colab_env
import os

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.28.8 requires python-dotenv==1.0.0, but you have python-dotenv 0.21.1 which is incompatible.
embedchain 0.1.113 requires python-dotenv<2.0.0,>=1.0.0, but you have python-dotenv 0.21.1 which is incompatible.
crewai-tools 0.38.1 requires click>=8.1.8, but you have click 8.1.7 which is incompatible.
crewai-tools 0.38.1 requires crewai>=0.105.0, but you have crewai 0.28.8 which is incompatible.
crewai-tools 0.38.1 requires embedchain>=0.1.114, but you have embedchain 0.1.113 which is incompatible.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [2]:
!pip install --upgrade crewai crewai_tools -q
!pip install transformers -U -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.1 which is incompatible.


In [ ]:
from crewai import Agent, Task, Crew, Process
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
import torch
from langchain.llms import HuggingFaceHub
import os
import colab_env

# Retrieve the token from the environment variable, if available
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

# If access_token_write is not set, fallback to using the HUGGINGFACEHUB_API_TOKEN env variable
if access_token_write is None:
    access_token_write = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
else:
   os.environ["MODEL_PROVIDER_HUGGINGFACE"] = "true"
   # Set the MODEL_PROVIDER environment variable to tell litellm to use Hugging Face Hub
   os.environ["MODEL_PROVIDER"] = "huggingface"

   print("THE HUGGINGFACE_ACCESS_TOKEN_WRITE IS WORKING ..... ")
   print('\n')

# If neither is set, prompt the user to enter the token manually
if access_token_write is None:
    access_token_write = input("Enter your Hugging Face API token: ")


# Initialize the HuggingFaceHub LLM using the token
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"  # Or your preferred model
hf_llm = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature": 0.1, "max_length": 200},
    huggingfacehub_api_token=access_token_write  # Pass the token here
)


# 2. Define Agents
researcher = Agent(
    role='Senior Research Analyst',
    goal='Conduct thorough research on a given topic using only the provided LLM.',
    backstory="You are a seasoned research analyst with a knack for finding key information.",
    allow_delegation=False,
    verbose=True,
    llm=hf_llm,  # Assign the Hugging Face Hub LLM
)


writer = Agent(
    role='Technical Writer',
    goal='Write a clear and concise tutorial based on research using only the provided LLM.',
    backstory="You are an expert technical writer, skilled in simplifying complex topics.",
    allow_delegation=False,
    verbose=True,
    llm=hf_llm,  # Assign the Hugging Face Hub LLM
)

# 3. Define Tasks
research_task = Task(
    description="Research the basics of crewAI, focusing on agents, tasks, and crews.",
    agent=researcher,
    expected_output="A concise summary of crewAI, agents, tasks, and crews.",
)

write_tutorial_task = Task(
    description="Write a short tutorial explaining crewAI using the research provided.",
    agent=writer,
    expected_output="A short tutorial explaining crewAI concepts.",
)

# 4. Form the Crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_tutorial_task],
    process=Process.sequential,
    verbose=True,
)

# 5. Kickoff the Crew
result = crew.kickoff()

# 6. Print the Result
print("\n\nTutorial using Hugging Face Hub LLM:\n")
print(result)